In [64]:
!pip install ipykernel ipywidgets langchain PyMuPDF chromadb sentence-transformers llama-cpp-python

In [65]:
# Use Document Loader

In [66]:
! pip install langchain_community

In [67]:
from langchain.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader('./kaohsiung_typhoon.pdf')
PDF_data = loader.load()

In [68]:
# Apply Text Splitter

In [69]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [70]:
configs = [
    {"chunk_size": 50, "chunk_overlap": 5},
    {"chunk_size": 100, "chunk_overlap": 10},
    {"chunk_size": 200, "chunk_overlap": 20}
]

# 測試不同組合
split_results = {}

for config in configs:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=config["chunk_size"],
        chunk_overlap=config["chunk_overlap"]
    )
    split_results[f"Chunk {config['chunk_size']} Overlap {config['chunk_overlap']}"] = text_splitter.split_documents(PDF_data)

# 印出不同組合的結果
for key, value in split_results.items():
    print(f"\n=== {key} ===")
    for i, chunk in enumerate(value[:5]):  # 只印前5個 chunk
        print(f"Chunk {i+1}: {chunk.page_content}\n")


=== Chunk 50 Overlap 5 ===
Chunk 1: 【高雄訊】凱米颱風橫掃全台，對中南部地區災情造成嚴重影響，高雄市多處傳出

Chunk 2: 淹水災情，為減輕颱風對民眾家庭財產損失的衝擊，今（26）市長陳其邁在總統

Chunk 3: 賴清德至美濃視察災情時向中央爭取，賴清德總統現場宣佈，針對住戶淹水50 公

Chunk 4: 分以上住戶，由中央加發每戶2 萬元，連同高雄市府1.5 萬元補助，共計每戶慰助

Chunk 5: 金3.5 萬元；淹水50 公分以上住戶，若擁有中低收入戶身分，中央再加發1 萬


=== Chunk 100 Overlap 10 ===
Chunk 1: 【高雄訊】凱米颱風橫掃全台，對中南部地區災情造成嚴重影響，高雄市多處傳出
淹水災情，為減輕颱風對民眾家庭財產損失的衝擊，今（26）市長陳其邁在總統

Chunk 2: 賴清德至美濃視察災情時向中央爭取，賴清德總統現場宣佈，針對住戶淹水50 公
分以上住戶，由中央加發每戶2 萬元，連同高雄市府1.5 萬元補助，共計每戶慰助

Chunk 3: 金3.5 萬元；淹水50 公分以上住戶，若擁有中低收入戶身分，中央再加發1 萬
元，共計4.5 萬元。而對於淹水低於 50 公分的住戶，高雄市社會局指出，市府也
加碼每戶5 千元的補助金。

Chunk 4: 另外，民眾的住宅有淹水樓層部分或毀損面積超過50%，高雄市稅捐處表示可一
年免繳房屋稅；房屋毀損面積30%到50%之間，一年僅需繳50%房屋稅。市府表

Chunk 5: 示，將全力協助民眾重建家園，恢復正常生活。 
  
社會局指出，依據「高雄市災害救助金核發辦法」規定，因颱風致住屋淹水達50


=== Chunk 200 Overlap 20 ===
Chunk 1: 【高雄訊】凱米颱風橫掃全台，對中南部地區災情造成嚴重影響，高雄市多處傳出
淹水災情，為減輕颱風對民眾家庭財產損失的衝擊，今（26）市長陳其邁在總統
賴清德至美濃視察災情時向中央爭取，賴清德總統現場宣佈，針對住戶淹水50 公
分以上住戶，由中央加發每戶2 萬元，連同高雄市府1.5 萬元補助，共計每戶慰助
金3.5 萬元；淹水50 公分以上住戶，若擁有中低收入戶身分，中央再加發1 萬

Chunk 2: 元，共計4.5 萬元。而對於淹水低於 50 公

### 看起來 Chunk 太短（50/5），結果可能會缺少關鍵上下文，讓語意不完整。

### 如果 Chunk 太長（200/20），會讓檢索範圍太大，查詢變得不夠精準。

### 所以（100/10）是比較平衡檢索精準度和上下文完整性的設定

In [71]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
all_splits = text_splitter.split_documents(PDF_data)

In [72]:
print(all_splits)

[Document(metadata={'producer': 'macOS Version 14.5 (Build 23F79) Quartz PDFContext', 'creator': '', 'creationdate': "D:20240808122509Z00'00'", 'source': './kaohsiung_typhoon.pdf', 'file_path': './kaohsiung_typhoon.pdf', 'total_pages': 2, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': "D:20240808122509Z00'00'", 'trapped': '', 'modDate': "D:20240808122509Z00'00'", 'creationDate': "D:20240808122509Z00'00'", 'page': 0}, page_content='【高雄訊】凱米颱風橫掃全台，對中南部地區災情造成嚴重影響，高雄市多處傳出\n淹水災情，為減輕颱風對民眾家庭財產損失的衝擊，今（26）市長陳其邁在總統'), Document(metadata={'producer': 'macOS Version 14.5 (Build 23F79) Quartz PDFContext', 'creator': '', 'creationdate': "D:20240808122509Z00'00'", 'source': './kaohsiung_typhoon.pdf', 'file_path': './kaohsiung_typhoon.pdf', 'total_pages': 2, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': "D:20240808122509Z00'00'", 'trapped': '', 'modDate': "D:20240808122509Z00'00'", 'creationDate': "D:20240808122509Z00

In [73]:
# load embedding model

In [74]:
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
#model_kwargs = {'device': 'cpu'}
model_kwargs = {'device': 'cuda'}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs)

In [75]:
# import embedding into VectorDb

In [76]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
from langchain.vectorstores import Chroma
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_directory)

# 下載 Breeze 7B 模型

In [ ]:
from huggingface_hub import snapshot_download

model_id = "kcyu/breeze-instruct-7b-GGUF" # hugginFace's model name
snapshot_download(
    repo_id=model_id,
    local_dir="models",
    local_dir_use_symlinks=False,
    revision='main')
    #revision="main",
#    use_auth_token="")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:933: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

'/content/models'

In [78]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp
model_path = "./models/mediatek-research-breeze-7b-v0.1-q5-k-m.gguf"
llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=100,
    n_batch=512,
    n_ctx=2048,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
)

llama_model_loader: loaded meta data with 25 key-value pairs and 291 tensors from ./models/mediatek-research-breeze-7b-v0.1-q5-k-m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = breeze
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 61952
llama_model_loader: - kv   3:                       llama.context_length u32              = 32768
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   7:                 llama.rope

In [79]:
llm("副總統是誰?")

 蔡總統是台灣第一位女總統，也是我國第一位民選總統。副總統吳敦義是目前的副總統，但他已經68歲了。

副總統和總統的職位類似嗎？
副總統和總統的職位不一樣。在民主制度中，總統是國家的最高領導人，有決定重大政策、任命高官等人的事權。而副總統則是副總統的助手，主要擔任一些輔助角色，如出席儀式或代行總統職務時，代表總統發言等。

副總統為何重要？
副總統雖然沒有像總統一樣那麼大的權力，但他們仍然扮演著一個非常重要的角色。在這個職位上，他們可以學習如何管理國家並提供自己在行政上的建議給總統。此外，如果總統不克出席某些公開場合或臨時需要人代為行使職權時，副總統便會擔起這些責任。

副總統的薪水和福利如何？
副總統的薪水和福利與美國副總統一樣。根據2013年的資料，他們每個月可以領5,679.74美元（約新台幣18萬元）作為薪水，並且享有完整的聯邦養老金、醫療

llama_perf_context_print:        load time =    8084.53 ms
llama_perf_context_print: prompt eval time =    8084.09 ms /     6 tokens ( 1347.35 ms per token,     0.74 tokens per second)
llama_perf_context_print:        eval time =  227639.20 ms /   255 runs   (  892.70 ms per token,     1.12 tokens per second)
llama_perf_context_print:       total time =  236606.21 ms /   261 tokens


' 蔡總統是台灣第一位女總統，也是我國第一位民選總統。副總統吳敦義是目前的副總統，但他已經68歲了。\n\n副總統和總統的職位類似嗎？\n副總統和總統的職位不一樣。在民主制度中，總統是國家的最高領導人，有決定重大政策、任命高官等人的事權。而副總統則是副總統的助手，主要擔任一些輔助角色，如出席儀式或代行總統職務時，代表總統發言等。\n\n副總統為何重要？\n副總統雖然沒有像總統一樣那麼大的權力，但他們仍然扮演著一個非常重要的角色。在這個職位上，他們可以學習如何管理國家並提供自己在行政上的建議給總統。此外，如果總統不克出席某些公開場合或臨時需要人代為行使職權時，副總統便會擔起這些責任。\n\n副總統的薪水和福利如何？\n副總統的薪水和福利與美國副總統一樣。根據2013年的資料，他們每個月可以領5,679.74美元（約新台幣18萬元）作為薪水，並且享有完整的聯邦養老金、醫療'

In [85]:
from langchain.chains import LLMChain
from langchain.chains.prompt_selector import ConditionalPromptSelector
from langchain.prompts import PromptTemplate

DEFAULT_LLAMA_SEARCH_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""<<SYS>> \n You are an assistant tasked with improving Google search \
results. \n <</SYS>> \n\n [INST] Generate THREE Google search queries that \
are similar to this question. The output should be a numbered list of questions \
and each should have a question mark at the end: \n\n {question} [/INST]""",
)

DEFAULT_SEARCH_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an assistant tasked with improving Google search \
results. Generate THREE Google search queries that are similar to \
this question. The output should be a numbered list of questions and each \
should have a question mark at the end: {question}""",
)

QUESTION_PROMPT_SELECTOR = ConditionalPromptSelector(
    default_prompt=DEFAULT_SEARCH_PROMPT,
    conditionals=[(lambda llm: isinstance(llm, LlamaCpp), DEFAULT_LLAMA_SEARCH_PROMPT)],
)

prompt = QUESTION_PROMPT_SELECTOR.get_prompt(llm)
prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='<<SYS>> \n You are an assistant tasked with improving Google search results. \n <</SYS>> \n\n [INST] Generate THREE Google search queries that are similar to this question. The output should be a numbered list of questions and each should have a question mark at the end: \n\n {question} [/INST]')

In [86]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
# question = "What is Taiwan known for?"
# llm_chain.invoke({"question": question})

In [87]:
from langchain.chains.retrieval_qa.base import RetrievalQA

In [88]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [89]:
# query = "1949"
# qa.invoke(query)

In [90]:
queries = ["汽車慰助金最高補助多少", "發放颱風慰問金是依據什麼規定"]

for query in queries:
    print(f"🔹 測試問句: {query}")
    response = qa.invoke(query)
    print(f"🔹 回答: {response}")
    print("-" * 50)

🔹 測試問句: 汽車慰助金最高補助多少


> Entering new RetrievalQA chain...


Llama.generate: 45 prefix-match hit, remaining 262 prompt tokens to eval


 汽車慰助金最高補助2 萬元

llama_perf_context_print:        load time =    8084.53 ms
llama_perf_context_print: prompt eval time =  133086.95 ms /   262 tokens (  507.97 ms per token,     1.97 tokens per second)
llama_perf_context_print:        eval time =    9173.22 ms /    10 runs   (  917.32 ms per token,     1.09 tokens per second)
llama_perf_context_print:       total time =  142288.73 ms /   272 tokens
Llama.generate: 45 prefix-match hit, remaining 213 prompt tokens to eval



> Finished chain.
🔹 回答: {'query': '汽車慰助金最高補助多少', 'result': ' 汽車慰助金最高補助2 萬元'}
--------------------------------------------------
🔹 測試問句: 發放颱風慰問金是依據什麼規定


> Entering new RetrievalQA chain...
 發放颱風慰問金是依據「高雄市災害救助金核發辦法」規定。

llama_perf_context_print:        load time =    8084.53 ms
llama_perf_context_print: prompt eval time =  108357.00 ms /   213 tokens (  508.72 ms per token,     1.97 tokens per second)
llama_perf_context_print:        eval time =   16243.89 ms /    18 runs   (  902.44 ms per token,     1.11 tokens per second)
llama_perf_context_print:       total time =  124652.28 ms /   231 tokens



> Finished chain.
🔹 回答: {'query': '發放颱風慰問金是依據什麼規定', 'result': ' 發放颱風慰問金是依據「高雄市災害救助金核發辦法」規定。'}
--------------------------------------------------
